In [ ]:
# Calling transformers from hugging face : WHere there are many trained transformer are there
!pip install transformers accelerate datasets #accelerate to speed up training process

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#fine tune this transformer for smaller datasets
# We are using t5: small-version
from datasets import load_dataset
mutlinews=load_dataset('multi_news', split='test', trust_remote_code=True) #Already Split

In [ ]:
mutlinews.to_pandas() # To see datsets how it look-like

document  \
0     GOP Eyes Gains As Voters In 11 States Pick Gov...   
1     \n \n \n \n UPDATE: 4/19/2001 Read Richard Met...   
2     It's the Golden State's latest version of the ...   
3     The seed for this crawl was a list of every ho...   
4     After a year in which liberals scored impressi...   
...                                                 ...   
5617  Tweet with a location \n \n You can add locati...   
5618  Loic Venance/AFP/Getty Images \n \n The awards...   
5619  (CNN) A federal criminal investigation into a ...   
5620  An archive of the public statements deleted by...   
5621  An apparently simple email about a holiday pot...   

                                                summary  
0     – It's a race for the governor's mansion in 11...  
1     – It turns out Facebook is only guilty of abou...  
2     – Not a big fan of Southern California? Neithe...  
3     – Why did Microsoft buy Nokia's phone business...  
4     – The Supreme Court is facing a docket of high...  
...                                                 ...  
5617  – The traditional end-of-summit group photo at...  
5618  – Sofia Coppola scored a historic victory at t...  
5619  – The duck boat sinking that killed 17 on a Mi...  
5620  – Note to tweeting politicians: Watch what you...  
5621  – "STOP THE MADNESS!!!!," "dear lord," and "I ...  

[5622 rows x 2 columns]

In [ ]:
#Becz we are training our model so we can't do web scrapping here not using them
# Here we are trying to extract the news from diff site as they have diff thought then we are using{quillbot} to summarize it.
# Each model has own embedding sys in which they take arguments :
#      Autotokenizer: processing of data as per 't5-small' required

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')#i/p & positional embedding both

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
# Testing & Train split
# train-part is to large therefor we are split test-part
multi_news=mutlinews.train_test_split(test_size=.2)

In [ ]:
#Processing Function
#Beyound-value is cutoof by truncation
prefix='summarize:'
def process_function(examples):
  inputs= [prefix+ doc for doc in examples['document']] # prefixed with "summarize" in doc
  model_inputs= tokenizer(inputs, max_length= 1024, truncation= True)
  labels= tokenizer(text= examples['summary'], max_length= 128, truncation= True)
  model_inputs['labels']= labels['input_ids'] #Useful for supervised learning input_ids are the numerical token representations of the text.
  return model_inputs

In [ ]:
tokenized_multi_news=multi_news.map(process_function, batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [ ]:
# DataCollatorForSeq2Seq specifically works with Seq2Seq models (like T5 or BART) and ensures correct padding for both input and target sequences.

# AutoModelForSeq2Seq is a general class for loading any pre-trained sequence-to-sequence model.LM:Language-model

#Seq2SeqTrainerArguments: elps set up training for models like T5 or BART

#Seq2SeqTrainer is a specialized version of Trainer for Seq2Seq models.
#It automates training, evaluation, and saving of sequence-to-sequence models.

In [ ]:
#setting-up Models
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collector= DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5_small') #data_collator
model=AutoModelForSeq2SeqLM.from_pretrained('t5-small', force_download=True)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#Hyperparam tuning
training_args= Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=.01, #Regularization-Technique
    save_total_limit=3, # Check-pt Saving
    num_train_epochs=10,
    fp16=True, #mixed-precision training
    report_to = 'none'
)

In [ ]:
## fp16 (Half-Precision Floating Point - 16-bit) enables mixed-precision training, where certain computations use 16-bit floating points (FP16)
# instead of the standard 32-bit (FP32)
## This prevents logging to platforms like WandB (Weights & Biases), TensorBoard, or MLflow.

In [ ]:
# Training-our model
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_multi_news['train'],
    eval_dataset= tokenized_multi_news['test'],
    #tokenizer= tokenizer,
    data_collator=data_collector,
)

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


TrainOutput(global_step=4500, training_loss=2.94947607421875, metrics={'train_runtime': 2823.709, 'train_samples_per_second': 15.926, 'train_steps_per_second': 1.594, 'total_flos': 1.217264162439168e+16, 'train_loss': 2.94947607421875, 'epoch': 10.0})

In [ ]:
text="""
Transformer is a neural network architecture used for performing machine learning tasks particularly in natural language processing (NLP) and computer vision. In 2017 Vaswani et al. published a paper ” Attention is All You Need” in which the transformers architecture was introduced. The article explores the architecture, workings and applications of transformers.

Need For Transformers Model in Machine Learning
Transformer Architecture is a model that uses self-attention to transform one whole sentence into a single sentence. This is useful where older models work step by step and it helps overcome the challenges seen in models like RNNs and LSTMs. Traditional models like RNNs (Recurrent Neural Networks) suffer from the vanishing gradient problem which leads to long-term memory loss. RNNs process text sequentially meaning they analyze words one at a time.

For example, in the sentence: “XYZ went to France in 2019 when there were no cases of COVID and there he met the president of that country” the word “that country” refers to “France”.


However RNN would struggle to link “that country” to “France” since it processes each word in sequence leading to losing context over long sentences. This limitation prevents RNNs from understanding the full meaning of the sentence.


While adding more memory cells in LSTMs (Long Short-Term Memory networks) helped address the vanishing gradient issue they still process words one by one. This sequential processing means LSTMs can’t analyze an entire sentence at once.

For instance the word “point” has different meanings in these two sentences:

“The needle has a sharp point.” (Point = Tip)
“It is not polite to point at people.” (Point = Gesture)
Traditional models struggle with this context dependence, whereas, Transformer model through its self-attention mechanism, processes the entire sentence in parallel addressing these issues and making it significantly more effective at understanding context.

Architecture and Working of Transformers
1. Positional Encoding
Unlike RNNs transformers lack an inherent understanding of word order since they process data in parallel. To solve this Positional Encodings are added to token embeddings providing information about the position of each token within a sequence.

2. Position-wise Feed-Forward Networks
The Feed-Forward Networks consist of two linear transformations with a ReLU activation. It is applied independently to each position in the sequence.

This transformation helps refine the encoded representation at each position.

3. Attention Mechanism
The attention mechanism allows transformers to determine which words in a sentence are most relevant to each other. This is done using a scaled dot-product attention approach:

1. Each word in a sequence is mapped to three vectors:

3. These scores determine how much attention each word should pay to others.

Multi-Head Attention
Instead of using a single attention mechanism transformers apply multi-head attention where multiple attention layers run in parallel. This enables the model to capture different types of relationships within the input.

4. Encoder-Decoder Architecture
The encoder-decoder structure is key to transformer models. The encoder processes the input sequence into a vector, while the decoder converts this vector back into a sequence. Each encoder and decoder layer includes self-attention and feed-forward layers. In the decoder, an encoder-decoder attention layer is added to focus on relevant parts of the input.
"""

In [ ]:
#Tokenization for our values
input_ids = tokenizer(text, max_length=128, truncation=True, return_tensors='pt').input_ids
#pytorch tensor #Its return whole dict but to get input_ids we need to explitily mention it
input_ids =input_ids.to('cuda') #All data transferes to GPU-Part
#beam search decoding with 5 beams, meaning the model explores multiple possible sequences and selects the most likely one
# It controls the number of beams in beam search for decoding

In [ ]:
import torch
with torch.no_grad():
  if model.device.type == 'cuda':
    output= model.generate(input_ids, max_length=139, num_beams=5)
summary_ids= output[0].tolist() # Summary encoded form

#decode
summary= tokenizer.decode(summary_ids, skip_special_tokens= True)
print(summary)

is a model that uses self-attention to transform one whole sentence into a single sentence. This is useful where older models work step by step and it helps overcome the challenges seen in models like RNNs and RNNs. The Transformer Architecture is a model that uses self-attention to transform one whole sentence into a single sentence. This is useful where older models work step by step and it helps overcome the challenges seen in models like RNNs and RNNs. This is useful where older models work step by step and it helps


In [ ]:
ref_summary="""
             Transformer is a neural network architecture used in machine learning, particularly in natural language processing and computer vision.
            Introduced in 2017 by Vaswani et al., Transformer uses self-attention to transform a whole sentence into a single sentence, overcoming challenges
             faced by traditional models like Recurrent Neural Networks (RNNs) and Long Short-Term Memory Networks (LSTMs). Transformers process the entire sentence in parallel, addressing the vanishing gradient problem and enabling better understanding of context.

Turbomotives have four main architectures: Positional Encoding, Position-wise Feed-Forward Networks, Attention Mechanism, and Encoder-Decoder Architecture. Positional Encoding adds information about the position of each token within a sequence, while Feed-Forward Networks refine the encoded representation at each position. Attention Mechanism determines the most relevant words in a sentence using a scaled dot-product approach. Multi-Head Attention allows the model to capture different types of relationships within the input.
""" #ref-summary from Quilblot to compare

In [ ]:
!pip install rouge

In [ ]:
# How's summary frompredicated to actual one we use rouge metrics
from rouge import Rouge
rouge= Rouge()
scores= rouge.get_scores(summary, ref_summary)
scores #{'rouge-1': {'r': value(recall), 'p': (precision), 'f': (f1-score)}}
#r-1:  long sequence matching with our :- 0.2  not so good
#r-2: check bigram tpow words amtching same
#r-l: refer to 1 but its regardless of the sequence


[{'rouge-1': {'r': 0.18691588785046728,
   'p': 0.5882352941176471,
   'f': 0.28368793960263566},
  'rouge-2': {'r': 0.06428571428571428,
   'p': 0.21428571428571427,
   'f': 0.09890109535080316},
  'rouge-l': {'r': 0.17757009345794392,
   'p': 0.5588235294117647,
   'f': 0.26950354243951513}}]

In [ ]:
#We can improve our model by providing it ore text
trainer.save_model()

In [ ]:
model.save_pretrained('Summarizer')#summarze_model saved